# Example: OWSLib extension for ESGF compute API

This notebook demonstrates a prototype of an [ESGF API](https://github.com/ESGF/esgf-compute-api) implementation based on [OWSLib](https://github.com/geopython/OWSLib) (WPS client) and it is also using a [PyWPS](https://pywps.org/) service for the WPS server part with a dummy implemenatation of an ESGF compute process (*Emu.subset*). The *Emu.subset* process is using *dask/xarray* to subset input netcdf files.

Please check the OWSLib code on GitHub for details:

* Code: https://github.com/bird-house/OWSLib/blob/esgfwps/owslib/esgfapi.py
* Tests: https://github.com/bird-house/OWSLib/blob/esgfwps/tests/test_esgfapi.py

You can compare this with notebook examples of the original ESGF compute interface: 

* https://github.com/ESGF/esgf-compute-api
* https://github.com/ESGF/esgf-compute-api/tree/master/examples


**TODO**: We can use PyWPS for WPS service definitions and build a seperate ESGF compute library for processing functionality. We can define an abstract PyWPS process class which can be used (subclassed) to define new ESGF-API processes.

See: 
* https://github.com/ESGF/esgf-compute-wps
* https://github.com/bird-house/emu/blob/esgfwps/emu/processes/wps_esgf_subset.py
* https://pywps.org/
* http://xarray.pydata.org/en/stable/dask.html

**TODO**: This prototype is incomplete. We just show a path how we can make use of existing solutions. All implementations need improvementes: OWSLib, OWSLib/esgfapi, PyWPS and ESGF-API itself. 

## WPS client OWSLib/esgfapi

In [ ]:
from owslib.wps import WebProcessingService

**ESGF Access Token**

**TODO**: use headers in WPS request to transport *api_key*. *api_key* can be handled either in pywps or security middleware like [Twitcher](https://twitcher.readthedocs.io/en/latest/).

In [ ]:
# ESGF Access Token
api_key = 'TOKEN'

# use headers
headers = {'api_key': api_key}

### Get Capabilities

**TODO**: Using dummy ESGF process from Emu. Having troubles to access ESGF compute demo service due to certificate issues.

See Emu Emu.subset process on [GitHub](https://github.com/bird-house/emu/blob/esgfwps/emu/processes/wps_esgf_subset.py).

In [ ]:
# client = WebProcessingService('https://aims2.llnl.gov/wps/', api_key=api_key, verify=False)
client = WebProcessingService('https://bovec.dkrz.de/ows/proxy/emu', headers=headers, verify=True)
# client = WebProcessingService('http://localhost:5000/wps', headers=headers, verify=True)

In [ ]:
for p in client.processes:
    print(p.identifier)

### Describe Process

In [ ]:
proc = client.describeprocess(
    'Emu.subset'  # 'CDAT.subset'
)
proc.identifier

In [ ]:
for inpt in proc.dataInputs:
    print(inpt.identifier, inpt.dataType)

### WPS Process Inputs

**Domain**

**TODO**: can we use WPS boundingbox to describe domain? Are there other OGC concepts we can use?

In [ ]:
from owslib.esgfapi import Domain, Dimension

In [ ]:
d0 = Domain([
    Dimension('time', 0, 1, crs='indices'),
    Dimension('lat', 40, 60, crs='values'),
    Dimension('lon', 0, 20, crs='values'),
])

In [ ]:
# show json
print(d0.json)

In [ ]:
# add domain to WPS inputs
inputs = [('domain', d0)]

**Variable**

In [ ]:
from owslib.esgfapi import Variable

**TODO**: Should we use the file transportation layer of PyWPS?

In [ ]:
# data files we want to process
files = [
    # OpenDAP, CORDEX EUR-44, tasmax, climate index SU (summer days)
    'http://opendap.knmi.nl/knmi/thredds/dodsC/CLIPC/gerics/climatesignalmaps/EUR-44/tasmax/su_python-2-7-6_GERICS_ens-multiModel-climatesignalmap-rcp85-EUR-44_yr_20700101-20991231_1971-2000.nc',
]


In [ ]:
# add them one by one to WPS inputs as Variable
for x in files:
    # variable=su (summer days climate index)
    inputs.append(('variable', Variable(uri=x, var_name='su')))  

In [ ]:
# show all WPS inputs
for inp in inputs:
    print(inp[1])

### Execute

In [ ]:
exec = client.execute(proc.identifier, inputs=inputs)

In [ ]:
exec.isComplete()

In [ ]:
exec.isSucceded()

**Outputs**

**TODO**: return multiple output files ... maybe using metalink.

See: https://github.com/bird-house/emu/issues/64

In [ ]:
# show the output ... well, in case of Emu WPS it is just a dummy.
for output in exec.processOutputs:
    print(output.identifier, output.reference)

**Plot Preview**

In [ ]:
from IPython.display import Image

In [ ]:
Image(exec.processOutputs[2].reference)

## Birdy

**TODO**: use birdy with owslib/esgfapi. Add tests for esgfapi functionality in birdy.

https://birdy.readthedocs.io/en/latest/